<a type="anchor"></a>
# **Proyecto MAC**
# **Clasificador con redes bayesianas**
Alumno: Victor Alberto Lopez Cardona

Exp: 747175

<a type="anchor" id="100"></a>
# **Contenido**

1.	[Introduccion](#1)
2.	[Requerimientos](#2)
3.	[Datos](#3)
4.	[Limpiar la informacion](#4)
5.	[Transformar la informacion en vectores](#5)
6.	[Entrenar el modelo](#6)
7.	[Pruebas](#7)
8.	[Conclusiones](#8)


# **1. Introduccion** <a type="anchor" id="1"></a>
[Contenido](#100)

## **Multinomial Naïve Bayes algorithm**

Con un modelo Multinomial Naïve Bayes, las muestras (vectores de características) representan las frecuencias con las que ciertos eventos han sido generados por un multinomial.(p1, . . . ,pn) 
donde pi es la probabilidad de que ocurra el evento i. Se prefiere usar el algoritmo Naïve Bayes multinomial en datos que se distribuyen multinomialmente. Es uno de los algoritmos estándar que se utiliza en la tipificación de categorización de texto.

## **Gaussian Naïve Bayes algorithm**


Cuando tenemos valores de atributos continuos, asumimos que los valores asociados con cada tipo se distribuyen de acuerdo con la distribución gaussiana o normal. Por ejemplo, suponga que los datos de entrenamiento contienen un atributo continuo x. Primero segmentamos los datos por tipo y luego calculamos la media y la varianza de x en cada tipo. Sea µi la media de los valores y sea σi la varianza de los valores asociados con el i-ésimo tipo. Supongamos que tenemos algún valor de observación xi. Entonces, la distribución de probabilidad de xi dado un tipo se puede calcular mediante la siguiente ecuación:


![Gaussian Naive Bayes algorithm](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQEWCcq1XtC1Yw20KWSHn2axYa7eY-a0T1TGtdVn5PvOpv9wW3FeA&s)

## **Bernoulli Naïve Bayes algorithm**

En el modelo de eventos multivariado de Bernoulli, las características son variables booleanas independientes (variables binarias) que describen entradas. Al igual que el modelo multinomial, este modelo también es popular para tareas de tipificación de documentos en las que se utilizan características de aparición de términos binarios en lugar de frecuencias de términos.

Para nuestro proyecto utlizaremos Multinomial Naive Bayes algorithm porque utlizamos la cantidad de veces que se repite una palabra en los mensajes como indicador de si un mensaje es SPAM o no, el algoritmo multinomial de bayes toma en cuenta clasificacion multi clase, por lo tanto funciona con muchos atributos, representaremos nuestro documento con un vector que contenga los atributos cuyo valor es la frecuencia de una palabra en el documento

# **2. Requerimientos** <a type="anchor" id="2"></a>
[Contenido](#100)

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


# **3. Datos** <a type="anchor" id="3"></a>
[Contenido](#100)

In [93]:
all_data = pd.read_csv('Spam_data',sep='\t',names=["type","message"])
all_data.head(10)

,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


Posibles indicadores?

In [94]:
all_data.groupby('type').describe()

message                                                               
       count unique                                                top freq
type                                                                       
ham     4825   4516                             Sorry, I'll call later   30
spam     747    653  Please call our customer service representativ...    4

In [95]:
all_data['length']=all_data['message'].apply(len)
all_data.length.describe()

count    5572.000000
mean       80.490309
std        59.944527
min         2.000000
25%        36.000000
50%        62.000000
75%       122.000000
max       910.000000
Name: length, dtype: float64

El mensaje mas largo consta de 910 caracteres, entonces...

La longitud del texto es un indicativo de si el mensaje es spam?

Ahora vamos a limpiar nuestra informacion

# **4. Limpiar la informacion** <a type="anchor" id="4"></a>
[Contenido](#100)


Para calcular todas las psoibilidades necesitamos tener un buen data frame, por lo que vamos a limpiar la informacion, vamos a remover las palabras que no nos aportan informacion y que mas se repiten, como "the", "a", "is"; La siguiente funcion se encarga de quitar esas palabras para todo el data frame.


In [96]:
import nltk
try:
    nltk.corpus.stopwords.words('english')
except:
    nltk.download('stopwords')
#remover letras que apocan poca info
def clean_data(mnsj):
    msg = [x for x in mnsj if x not in string.punctuation]
    msg = ''.join(msg)
    clean_msg = [x for x in msg.split() if x.lower not in nltk.corpus.stopwords.words('english')]
    return clean_msg

#all_data['message'].apply(clean_data)


# **5. Transformar la informacion en vectores** <a type="anchor" id="5"></a>
[Contenido](#100)

Actualmente tenemos un data frame con listas que contienen el mensaje, para poder vectorizar la informacion, vamos a hacer uso de la libreria sklearn y usaremos la funcion countvectorizer para obtener una matriz de tokens donde podremos ver cuantas veces se repite una palabra en cada mensaje

In [97]:
vector = CountVectorizer(analyzer=clean_data).fit(all_data['message'])
df_vector = vector.transform(all_data['message'])
frecuencias=TfidfTransformer().fit(df_vector)
frecuencias_data_frame=frecuencias.transform(df_vector)
print(frecuencias_data_frame.shape)


(5572, 11747)


# **6. Entrenar el modelo** <a type="anchor" id="6"></a>
[Contenido](#100)

In [98]:
modelo = MultinomialNB().fit(frecuencias_data_frame,all_data['type'])



# **7. Pruebas** <a type="anchor" id="7"></a>
[Contenido](#100)

Vamos a probar con todo nuestro data frame ya vectorizado, y el modelo entrenado nos va a clasificar los mensajes, el modelo nos va a regresar una lista con los valores clasificados

In [99]:
resultados = modelo.predict(frecuencias_data_frame)
print(resultados)

['ham' 'ham' 'spam' ... 'ham' 'ham' 'ham']


In [100]:
for i in range(0,50):
    if resultados[i] == "ham":
        print(f"El mensaje {all_data['message'][i]} ha sido detectado como seguro")
    elif resultados[i] == "spam":
        print(f"El mensaje {all_data['message'][i]} ha sido detectado como SPAM")

El mensaje Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat... ha sido detectado como seguro
El mensaje Ok lar... Joking wif u oni... ha sido detectado como seguro
El mensaje Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's ha sido detectado como SPAM
El mensaje U dun say so early hor... U c already then say... ha sido detectado como seguro
El mensaje Nah I don't think he goes to usf, he lives around here though ha sido detectado como seguro
El mensaje FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv ha sido detectado como seguro
El mensaje Even my brother is not like to speak with me. They treat me like aids patent. ha sido detectado como seguro
El mensaje As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam

Checar la precision

In [101]:

puntaje = accuracy_score(all_data['type'], resultados)
print(f'Precision: {puntaje*100}')


Precision: 97.21823402727925


# **8. Conclusiones** <a type="anchor" id="8"></a>
[Contenido](#100)

El objetivo del proyecto es hacer uso de redes bayesianas ingenuas para clasificar un mensaje como SPAM o como mensaje seguro, para hacer esto podemos hacer uso de 3 distintos algoritmos, que son el Multinomial, Gaussiano, y de Bernoulli, para este proyecto se opto por utilizar el algoritmo multinomial ya que para nuestro set de información que es un texto, este algoritmo funciona mejor.

El proyecto logra clasificar los mensajes como SPAM o mensaje seguro con un 97% de precisión aproximadamente.
Como áreas de mejora o trabajo futuro, podemos normalizar la información que tenemos, hay algunos mensajes que contienen una palabra mal escrita y podemos detectar esta palabra y corregirla para tener una tabla de frecuencias correcta en nuestro proyecto, esto nos llevara a un modelo de predicción mas preciso.

En el código podemos observar que tenemos una función clean_data(), en esta función estamos eliminando palabras que no nos aportan información valiosa al mensaje, por lo que las eliminamos por cada uno de los mensajes que tenemos, esto mejoro la precisión de nuestro modelo en un 30%, ya que la mayoría de las palabras que se repetían en los mensajes eran “the”, “a”, is“, y el modelo identificaba erróneamente los mensajes que contenían SPAM.
